importing required packages

In [2]:
import pandas as pd
import numpy as np
import time
import os
import pickle
from imdb import Cinemagoer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [70]:
movies_df1 = pd.read_csv(r"C:\Users\P PAVAN REDDY\Documents\My Files\Projects\Movie Recommendation system\Data\movies_df1.csv")
movies_df2 = pd.read_csv(r"C:\Users\P PAVAN REDDY\Documents\My Files\Projects\Movie Recommendation system\Data\movies_df2.csv")

In [71]:
movies_df1.info()#.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3631 entries, 0 to 3630
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Title                 3631 non-null   object
 1   release_year          3631 non-null   int64 
 2   imdb_id               3631 non-null   object
 3   poster                3631 non-null   object
 4   tmdb_id               3631 non-null   int64 
 5   cast                  3631 non-null   object
 6   director              3631 non-null   object
 7   genres                3631 non-null   object
 8   production_companies  3631 non-null   object
dtypes: int64(2), object(7)
memory usage: 255.4+ KB


In [72]:
movies_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2515 entries, 0 to 2514
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Title                 2515 non-null   object
 1   release_year          2515 non-null   int64 
 2   imdb_id               2515 non-null   object
 3   poster                2515 non-null   object
 4   tmdb_id               2515 non-null   int64 
 5   cast                  2515 non-null   object
 6   director              2515 non-null   object
 7   genres                2515 non-null   object
 8   production_companies  2515 non-null   object
dtypes: int64(2), object(7)
memory usage: 177.0+ KB


In [73]:
#let's concat these two dataframes
movies = pd.concat([movies_df1, movies_df2], ignore_index=True)

In [74]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Title                 6146 non-null   object
 1   release_year          6146 non-null   int64 
 2   imdb_id               6146 non-null   object
 3   poster                6146 non-null   object
 4   tmdb_id               6146 non-null   int64 
 5   cast                  6146 non-null   object
 6   director              6146 non-null   object
 7   genres                6146 non-null   object
 8   production_companies  6146 non-null   object
dtypes: int64(2), object(7)
memory usage: 432.3+ KB


In [79]:
#making columns names into lower case
movies.columns = list(map(str.lower, movies.columns))

In [81]:
movies

,title,release_year,imdb_id,poster,tmdb_id,cast,director,genres,production_companies
0,29th Street,1991,tt0101252,https://m.media-amazon.com/images/M/MV5BMWY2MT...,43267,"['Anthony LaPaglia', 'Danny Aiello', 'Lainie K...",George Gallo,['Comedy'],['20th Century Fox']
1,The Addams Family,1991,tt0101272,https://m.media-amazon.com/images/M/MV5BODc1Nm...,2907,"['Raúl Juliá', 'Anjelica Huston', 'Christopher...",Barry Sonnenfeld,"['Comedy', 'Fantasy']","['Scott Rudin Productions', 'Orion Pictures', ..."
2,All I Want for Christmas,1991,tt0101301,https://m.media-amazon.com/images/M/MV5BMTk3Mj...,13644,"['Harley Jane Kozak', 'Jamey Sheridan', 'Thora...",Rob Lieberman,"['Family', 'Drama', 'Comedy']",['Paramount']
3,American Kickboxer,1991,tt0101325,https://m.media-amazon.com/images/M/MV5BM2E2Nm...,110830,"['John Barrett', 'Keith Vitali', 'Terry Norton...",Frans Nel,['Action'],"['Global Pictures', 'Distant Horizon', 'Cannon..."
4,An American Tail: Fievel Goes West,1991,tt0101329,https://m.media-amazon.com/images/M/MV5BZmUyOT...,10380,"['Phillip Glasser', 'James Stewart', 'Erica Yo...",Simon Wells,"['Adventure', 'Animation', 'Family', 'Western']","['Amblin Entertainment', 'Universal Pictures',..."
...,...,...,...,...,...,...,...,...,...
6147,Iron Man 3,2013,tt1300854,https://m.media-amazon.com/images/M/MV5BMjE5Mz...,68721,"['Robert Downey Jr.', 'Gwyneth Paltrow', 'Don ...",Shane Black,"['Action', 'Adventure', 'Science Fiction']",['Marvel Studios']
6148,Avatar: The Way of Water,2022,tt1630029,https://m.media-amazon.com/images/M/MV5BYjhiNj...,76600,"['Sam Worthington', 'Zoe Saldaña', 'Sigourney ...",James Cameron,"['Science Fiction', 'Adventure', 'Action']","['20th Century Studios', 'Lightstorm Entertain..."
6149,Star Wars: The Force Awakens,2015,tt2488496,https://m.media-amazon.com/images/M/MV5BOTAzOD...,140607,"['Harrison Ford', 'Mark Hamill', 'Carrie Fishe...",J.J. Abrams,"['Adventure', 'Action', 'Science Fiction']","['Lucasfilm Ltd.', 'Bad Robot']"
6150,Star Wars: The Last Jedi,2017,tt2527336,https://m.media-amazon.com/images/M/MV5BMjQ1Mz...,181808,"['Mark Hamill', 'Carrie Fisher', 'Adam Driver'...",Rian Johnson,"['Adventure', 'Action', 'Science Fiction']",['Lucasfilm Ltd.']


In [82]:
for feature in movies.columns:
    print("{}-> \n".format(feature), movies[feature].value_counts(), "\n")

title-> 
 Payback                         3
No Escape                       2
Hope Springs                    2
The Kid                         2
Venom                           2
                               ..
Passion of Mind                 1
Panic                           1
Our Song                        1
The Original Kings of Comedy    1
Furious 7                       1
Name: title, Length: 6060, dtype: int64 

release_year-> 
 2021    278
1996    259
1997    259
2009    237
1995    228
2020    222
1998    214
2019    207
2001    207
1999    204
2018    199
2022    197
1994    197
2014    196
2000    195
1993    195
2010    195
1991    194
2008    193
2003    193
2013    188
2007    187
1992    186
2017    186
2012    185
2011    165
2006    164
2004    161
2016    156
2005    148
2015    147
2002     10
Name: release_year, dtype: int64 

imdb_id-> 
 tt0962736     2
tt3155342     2
tt10832274    2
tt0101252     1
tt0431021     1
             ..
tt0195945     1
tt0156841     

In [46]:
#df[df['results'].map(len)
sum(movies['cast'] == 'unknown')

24

In [47]:
sum(movies['director'] == 'unknown')

3

In [48]:
sum(movies['genres'] == 'unknown')

26

In [49]:
sum(movies['genres']=='[]')   #19 '[]'

19

<b>Issues in data
* 'cast' -> where ever cast is missing we have 'unknown' string init
* 'director' -> where ever director is missing we have 'unknown' string init
* 'genres' -> where ever genres is missing we have empty list '[]' init or have string 'unknown'      
* the datahave duplicate rows                                         
* we need to remove all special characters in columns 
* 'production_company' -> has so many '[]' init where it is missing   
* remove white spaces in between words of single name like 'Tony Stark' -> 'TonyStark'

* Let's not drop the rows with missing value ('[]'), instead let's replace it with 'unknown'
* and collect the missing data using imdb api

In [50]:
movies_copy = movies.copy()

genres

In [51]:
sum(movies_copy['genres']=='[]')   #19 '[]'

19

In [52]:
movies_copy['genres'].replace('[]', 'unknown', inplace=True)

In [53]:
sum(movies_copy['genres']=='[]') #0

0

duplicate columns

In [54]:
#As we know id is the unique we can use it as reference
sum(movies_copy['tmdb_id'].duplicated())

3

In [55]:
#dropping columns with duplicate ids
ids_dup_index = movies_copy[movies_copy['tmdb_id'].duplicated()].index
movies_copy.drop(ids_dup_index, inplace=True)

movies_copy.reset_index( drop=True, inplace=True)

In [56]:
sum(movies_copy['tmdb_id'].duplicated())  #we have dropped duplicated values

0

In [57]:
sum(movies_copy['tmdb_id'].value_counts() > 1) 

0

* But there may be diffrerent movies with same names, released in different years 

In [58]:
#movies_copy['release_year'] = movies_copy['release_year'].apply(lambda x : str(x)[:-2]  if (len(str(x))>4) else x)

production_companies

In [59]:
movies_copy['production_companies'].value_counts().index[0] #most frequent value

'[]'

In [60]:
movies_copy['production_companies'].value_counts()[0]   #we can see we have [] in 425 rows

241

In [61]:
#replacing '[]' with 'unknown'
movies_copy['production_companies'].replace('[]', 'unknown', inplace=True)

In [62]:
sum(movies_copy['production_companies'] == '[]')

0

In [63]:
movies_copy['production_companies'].value_counts().index[0] #most frequent value

'unknown'

# let's collect missing data using imdb api library

In [64]:
cast_unknown = movies_copy[movies_copy['cast'] == 'unknown']['imdb_id'].to_dict()
len(cast_unknown)

24

In [67]:
cast_unknown

{550: 'tt0108323',
 1088: 'tt0116310',
 1619: 'tt0208261',
 1674: 'tt0181786',
 2027: 'tt0188028',
 2081: 'tt0240885',
 2094: 'tt0240912',
 2300: 'tt0143861',
 2333: 'tt0357309',
 3034: 'tt0972541',
 3344: 'tt1198405',
 3419: 'tt1239427',
 3698: 'tt1548865',
 4031: 'tt1430625',
 4102: 'tt2125490',
 4119: 'tt1618399',
 4473: 'tt3450654',
 4538: 'tt3501904',
 4680: 'tt2159988',
 4722: 'tt3304610',
 5140: 'tt8150132',
 5433: 'tt9097270',
 5719: 'tt12672720',
 5813: 'tt9124868'}

Updating cast

In [ ]:
# create an instance of the Cinemagoer class
ia = Cinemagoer()

cast_matrix = []
for ids in cast_unknown.values():
    the_matrix = ia.get_movie(ids[2:])
    cast_matrix.append(the_matrix)

KeyboardInterrupt: 

In [348]:
len(cast_matrix)

24

In [349]:
cast_fetched = []

for record in range(len(cast_matrix)):
    
    if 'cast' in cast_matrix[record]:
        if len(cast_matrix[record]['cast'])>=6:
            cast_fetched.append([cast_matrix[record]['cast'][0]['name'],
                         cast_matrix[record]['cast'][1]['name'],
                         cast_matrix[record]['cast'][2]['name'],
                         cast_matrix[record]['cast'][3]['name'],
                         cast_matrix[record]['cast'][4]['name'],
                         cast_matrix[record]['cast'][5]['name']])
            
        elif len(cast_matrix[record]['cast'])==5:
            cast_fetched.append([cast_matrix[record]['cast'][0]['name'],
                         cast_matrix[record]['cast'][1]['name'],
                         cast_matrix[record]['cast'][2]['name'],
                         cast_matrix[record]['cast'][3]['name'],
                         cast_matrix[record]['cast'][4]['name']])
            
        elif len(cast_matrix[record]['cast'])==4:
            cast_fetched.append([cast_matrix[record]['cast'][0]['name'],
                         cast_matrix[record]['cast'][1]['name'],
                         cast_matrix[record]['cast'][2]['name'],
                         cast_matrix[record]['cast'][3]['name']])
            
        elif len(cast_matrix[record]['cast'])==3:
            cast_fetched.append([cast_matrix[record]['cast'][0]['name'],
                         cast_matrix[record]['cast'][1]['name'],
                         cast_matrix[record]['cast'][2]['name']])
            
        elif len(cast_matrix[record]['cast'])==2:
            cast_fetched.append([cast_matrix[record]['cast'][0]['name'],
                         cast_matrix[record]['cast'][1]['name']])
            
        elif len(cast_matrix[record]['cast'])==1:
            cast_fetched.append([cast_matrix[record]['cast'][0]['name']])
            
            
        else:
            cast_fetched.append('unknown')
            
    else:
        cast_fetched.append('unknown')

In [350]:
len(cast_fetched)

24

In [351]:
#updating cast_unknown dict with fetched values
for key, value in cast_unknown.items():  
    cast_unknown[key] = cast_fetched.pop(0)

In [353]:
len(cast_unknown)

24

* updating rows in cast column where it has 'unknown' value, with fetched values

In [354]:
movies_copy["cast"].update(cast_unknown)

In [357]:
movies_copy[movies_copy['cast'] == 'unknown']  #only two left, let's drop them

,title,release_year,imdb_id,poster,tmdb_id,cast,director,genres,production_companies
3698,12th & Delaware,2010,tt1548865,https://m.media-amazon.com/images/M/MV5BMTQwNj...,44806,unknown,Heidi Ewing,['Documentary'],['Loki Films']
5140,The Guardians,2018,tt8150132,https://m.media-amazon.com/images/M/MV5BNDllOG...,509910,unknown,Ben Crosbie,['Documentary'],['Eidolon Films']


In [362]:
movies_copy.drop(movies_copy[movies_copy['cast'] == 'unknown'].index, inplace=True)
movies_copy.reset_index(drop=True, inplace=True)

updating director

In [372]:
movies_copy[movies_copy['director'] == 'unknown']

,title,release_year,imdb_id,poster,tmdb_id,cast,director,genres,production_companies
2593,Off the Lip,2004,tt0223828,https://m.media-amazon.com/images/M/MV5BMTQ0NT...,279709,"['Marguerite Moreau', 'Adam Scott', 'Mackenzie...",unknown,"['Adventure', 'Comedy']",unknown
3159,Sarah Landon and the Paranormal Hour,2007,tt0990376,https://m.media-amazon.com/images/M/MV5BMTkwNz...,23327,"['Rissa Walters', 'Michael Silva', 'Jack Pine-...",unknown,"['Family', 'Horror', 'Mystery', 'Thriller']",unknown
5811,Rollers,2021,tt9124868,https://m.media-amazon.com/images/M/MV5BMjg5MD...,842170,"[Kevin Bigley, Vicky Jeudy, Meredith Thomas, K...",unknown,['Comedy'],unknown


In [371]:
director_unknown = movies_copy[movies_copy['director'] == 'unknown']['imdb_id'].to_dict()
director_unknown

{2593: 'tt0223828', 3159: 'tt0990376', 5811: 'tt9124868'}

In [ ]:
# create an instance of the Cinemagoer class
ia = Cinemagoer()

director_matrix = []
for ids in director_unknown.values():
    the_matrix1 = ia.get_movie(ids[2:])
    director_matrix.append(the_matrix1)

In [395]:
director_fetch = []

for record in director_matrix:
    director_fetch.append(record['directors'][0]['name'])

In [396]:
director_fetch

['Robert Mickelson', 'Lisa Comrie', 'Isaiah Smallman']

In [403]:
#updating director_unknown dict with fetched values
for key, value in director_unknown.items():  
    director_unknown[key] = director_fetch.pop(0)

In [405]:
director_unknown

{2593: 'Robert Mickelson', 3159: 'Lisa Comrie', 5811: 'Isaiah Smallman'}

* updating rows in director column where it has 'unknown' value, with fetched values

In [406]:
movies_copy["director"].update(director_unknown)

In [407]:
movies_copy[movies_copy['director'] == 'unknown']

,title,release_year,imdb_id,poster,tmdb_id,cast,director,genres,production_companies


updating genres 

In [409]:
len(movies_copy[movies_copy['genres'] == 'unknown'])

45

In [416]:
genres_unknown = movies_copy[movies_copy['genres'] == 'unknown']['imdb_id'].to_dict()
genres_unknown

{667: 'tt0497630',
 1095: 'tt0119138',
 1304: 'tt0237072',
 1323: 'tt0119001',
 1608: 'tt0199685',
 1617: 'tt0165857',
 1629: 'tt0127020',
 1752: 'tt0159277',
 1827: 'tt0140352',
 1834: 'tt0160672',
 1855: 'tt0130121',
 2187: 'tt0264559',
 2219: 'tt0174732',
 2278: 'tt0250636',
 2315: 'tt0234829',
 2336: 'tt0281398',
 2365: 'tt0357470',
 2401: 'tt0309530',
 2408: 'tt0368721',
 2426: 'tt0338094',
 2438: 'tt0286716',
 2439: 'tt0308383',
 2472: 'tt0242653',
 2473: 'tt0288045',
 2493: 'tt0316396',
 2520: 'tt0314353',
 2521: 'tt0181852',
 2524: 'tt0328538',
 2525: 'tt0343095',
 2552: 'tt0343660',
 2554: 'tt0398872',
 2565: 'tt0360009',
 2580: 'tt0327679',
 2598: 'tt0298148',
 2636: 'tt0366174',
 2643: 'tt0324554',
 2799: 'tt0412922',
 3257: 'tt0893325',
 3585: 'tt1172994',
 3588: 'tt0829427',
 3632: 'tt6622356',
 4472: 'tt3450654',
 5180: 'tt10806200',
 5502: 'tt12838350',
 5717: 'tt12672720'}

In [ ]:
ia = Cinemagoer()

genres_matrix = []
for ids in genres_unknown.values()[]:
    the_matrix2 = ia.get_movie(ids[2:])
    genres_matrix.append(the_matrix2)

In [423]:
len(genres_matrix)

45

In [428]:
genres_fetch = []
count=0

for record in genres_matrix:
    count+=1
    genres_fetch.append(record['genres'])

In [429]:
genres_fetch

[['Drama', 'Music'],
 ['Drama'],
 ['Comedy', 'Horror'],
 ['Drama', 'Romance'],
 ['Action', 'Adventure', 'Comedy', 'Crime', 'Thriller'],
 ['Drama'],
 ['Drama'],
 ['Drama'],
 ['Biography', 'Drama', 'Thriller'],
 ['Crime', 'Drama'],
 ['Comedy', 'Crime', 'Romance'],
 ['Adventure', 'Comedy', 'Mystery'],
 ['Comedy', 'Romance'],
 ['Drama', 'Mystery', 'Thriller'],
 ['Comedy', 'Drama', 'Romance', 'Sport'],
 ['Comedy', 'Drama', 'Sci-Fi'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy', 'Romance'],
 ['Documentary'],
 ['Comedy', 'Family', 'Fantasy', 'Horror', 'Mystery'],
 ['Action', 'Sci-Fi'],
 ['Drama', 'Romance', 'Thriller'],
 ['Action', 'Sci-Fi'],
 ['Action', 'Comedy', 'Fantasy'],
 ['Adventure', 'Family', 'Fantasy', 'Romance'],
 ['Action', 'Drama', 'Thriller', 'War'],
 ['Action', 'Sci-Fi'],
 ['Drama'],
 ['Drama'],
 ['Comedy', 'Drama', 'Romance'],
 ['Animation', 'Adventure', 'Comedy', 'Family', 'Fantasy'],
 ['Action', 'Crime', 'Drama', 'Mystery', 'Thriller'],
 ['Comedy', 'Family', 'Fantasy', 'Roman

In [431]:
#updating genres_unknown dict with fetched values
for key, value in genres_unknown.items():  
    genres_unknown[key] = genres_fetch.pop(0)

In [432]:
genres_unknown

{667: ['Drama', 'Music'],
 1095: ['Drama'],
 1304: ['Comedy', 'Horror'],
 1323: ['Drama', 'Romance'],
 1608: ['Action', 'Adventure', 'Comedy', 'Crime', 'Thriller'],
 1617: ['Drama'],
 1629: ['Drama'],
 1752: ['Drama'],
 1827: ['Biography', 'Drama', 'Thriller'],
 1834: ['Crime', 'Drama'],
 1855: ['Comedy', 'Crime', 'Romance'],
 2187: ['Adventure', 'Comedy', 'Mystery'],
 2219: ['Comedy', 'Romance'],
 2278: ['Drama', 'Mystery', 'Thriller'],
 2315: ['Comedy', 'Drama', 'Romance', 'Sport'],
 2336: ['Comedy', 'Drama', 'Sci-Fi'],
 2365: ['Comedy', 'Drama', 'Romance'],
 2401: ['Comedy', 'Romance'],
 2408: ['Documentary'],
 2426: ['Comedy', 'Family', 'Fantasy', 'Horror', 'Mystery'],
 2438: ['Action', 'Sci-Fi'],
 2439: ['Drama', 'Romance', 'Thriller'],
 2472: ['Action', 'Sci-Fi'],
 2473: ['Action', 'Comedy', 'Fantasy'],
 2493: ['Adventure', 'Family', 'Fantasy', 'Romance'],
 2520: ['Action', 'Drama', 'Thriller', 'War'],
 2521: ['Action', 'Sci-Fi'],
 2524: ['Drama'],
 2525: ['Drama'],
 2552: ['Come

* updating rows in genres column where it has 'unknown' value, with fetched values

In [ ]:
movies_copy['genres'].update(genres_unknown)

In [440]:
movies_copy[movies_copy['genres'] == 'unknown']

,title,release_year,imdb_id,poster,tmdb_id,cast,director,genres,production_companies


updating production_companies

In [171]:
len(movies_copy[movies_copy['production_companies'] == 'unknown'])

267

In [172]:
prdc_unknown = movies_copy[movies_copy['production_companies'] == 'unknown']['imdb_id'].to_dict()
len(prdc_unknown)

267

In [35]:
prdc_matrix = []
count=0

In [ ]:
# imdb api library is taking too much time so let's use imdb api from rapid api website

for i in list(prdc_unknown.values()):
    
    url = "https://imdb8.p.rapidapi.com/title/get-company-credits"

    querystring = {"tconst":"{}".format(i)}

    headers = {
        "X-RapidAPI-Key": "ef44d30bedmsh7d2930aa0e90321p11240djsnf550adfe0503",
        "X-RapidAPI-Host": "imdb8.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    prdc_matrix.append(response.json())
    count+=1

* i converted prdc_matrix matrix into a dataframe called 'prdc' and named the column as 'jsons'

In [179]:
pcs = [] #for collecting names
for record in prdc['jsons']:
    
    if record != 'no result':
        if record[0]['id'] == 'production':
            
            pc_temp = []
            for i in range(len(record[0]['section']['items'])):
                
                pc_temp.append(record[0]['section']['items'][i]['rowTitle'])
            
            pcs.append(pc_temp)
                
                
            
        else:
            pcs.append('unknown')
    else:
        pcs.append('unknown')

In [180]:
len(pcs) #this contains extrcted production companies

267

In [181]:
#updating in prdc_unknown
for key, value in prdc_unknown.items():  
    prdc_unknown[key] = pcs.pop(0)

In [182]:
len(prdc_unknown)

267

* updating rows in production_companies column where it has 'unknown' value, with fetched values

In [ ]:
movies_copy['production_companies'].update(prdc_unknown)

In [68]:
 # no we have only 23 rows unknown let's leave them
len(movies_copy[movies_copy['production_companies'] == 'unknown']) 

267

# Text cleaning

cleaning generes, cast, director, production_companies

* removing special characters in between entities/names in columns genres, cast except ',' which we'll remove later

In [232]:
movies_copy = pd.read_csv(r"C:\Users\P PAVAN REDDY\Documents\My Files\Projects\Movie Recommendation system\movies_cleaned.csv")
movies_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6141 entries, 0 to 6140
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   title                 6141 non-null   object
 1   release_year          6141 non-null   int64 
 2   imdb_id               6141 non-null   object
 3   poster                6141 non-null   object
 4   tmdb_id               6141 non-null   int64 
 5   cast                  6141 non-null   object
 6   director              6141 non-null   object
 7   genres                6141 non-null   object
 8   production_companies  6141 non-null   object
dtypes: int64(2), object(7)
memory usage: 431.9+ KB


In [233]:
movies_copy = pd.concat([movies_copy, top_missing], ignore_index=True)

In [234]:
#cast
movies_copy['cast'] = movies_copy['cast'].apply(lambda s : s.translate({ord(i): '' for i in ".[]'\""}))

In [235]:
movies_copy['cast'][0]

'AnthonyLaPaglia DannyAiello LainieKazan FrankPesce RobertForster RonKarabatsos'

In [236]:
#genres
movies_copy['genres'] = movies_copy['genres'].apply(lambda s : s.translate({ord(i): '' for i in ".[]'\""}))

In [237]:
movies_copy['genres'].sample(5)

3987    Thriller Crime Mystery
2477     Drama History Romance
591              Comedy Family
5813       Crime Drama Mystery
3118                     Drama
Name: genres, dtype: object

In [238]:
#director 
movies_copy['director'] = movies_copy['director'].apply(lambda s : s.translate({ord(i): '' for i in ".[]'\""}))

In [239]:
movies_copy.iloc[0]['director']

'GeorgeGallo'

In [240]:
#production_companies
movies_copy['production_companies'] = movies_copy['production_companies'].apply(lambda s : s.translate({ord(i): '' for i in ".[]'\""}))

Now we got cleaned data which is suitable for displaying in the website so, let's export it into our database

In [241]:
#after removing special characters 
movies_copy[['genres', 'cast', 'director', 'production_companies']].tail()

,genres,cast,director,production_companies
6142,"Action, Adventure, Science Fiction","Robert Downey Jr, Gwyneth Paltrow, Don Cheadle...",Shane Black,Marvel Studios
6143,"Science Fiction, Adventure, Action","Sam Worthington, Zoe Saldaña, Sigourney Weaver...",James Cameron,"20th Century Studios, Lightstorm Entertainment"
6144,"Adventure, Action, Science Fiction","Harrison Ford, Mark Hamill, Carrie Fisher, Ada...",JJ Abrams,"Lucasfilm Ltd, Bad Robot"
6145,"Adventure, Action, Science Fiction","Mark Hamill, Carrie Fisher, Adam Driver, Daisy...",Rian Johnson,Lucasfilm Ltd
6146,"Action, Thriller, Crime","Vin Diesel, Paul Walker, Jason Statham, Michel...",James Wan,"Original Film, One Race, Universal Pictures"


In [242]:
#movies_copy.to_csv("movies_display.csv", index=False)

remove white spaces in between words of each entity/name

In [ ]:
#cast
movies_copy['cast'] = movies_copy['cast'].apply(lambda x : x.replace(' ', ''))
movies_copy['cast'] = movies_copy['cast'].apply(lambda x : x.replace(',', ' '))
#genres
movies_copy['genres'] = movies_copy['genres'].apply(lambda x : x.replace(' ', ''))
movies_copy['genres'] = movies_copy['genres'].apply(lambda x : x.replace(',', ' '))
#director
movies_copy['director'] = movies_copy['director'].apply(lambda x : x.replace(' ', ''))
movies_copy['director'] = movies_copy['director'].apply(lambda x : x.replace(',', ' '))
#production_companies
movies_copy['production_companies'] = movies_copy['production_companies'].apply(lambda x : x.replace(' ', ''))
movies_copy['production_companies'] = movies_copy['production_companies'].apply(lambda x : x.replace(',', ' '))

In [244]:
movies_copy[['cast', 'genres', 'director', 'production_companies']].tail()  #no white spaces

,cast,genres,director,production_companies
6142,RobertDowneyJr GwynethPaltrow DonCheadle GuyPe...,Action Adventure ScienceFiction,ShaneBlack,MarvelStudios
6143,SamWorthington ZoeSaldaña SigourneyWeaver Step...,ScienceFiction Adventure Action,JamesCameron,20thCenturyStudios LightstormEntertainment
6144,HarrisonFord MarkHamill CarrieFisher AdamDrive...,Adventure Action ScienceFiction,JJAbrams,LucasfilmLtd BadRobot
6145,MarkHamill CarrieFisher AdamDriver DaisyRidley...,Adventure Action ScienceFiction,RianJohnson,LucasfilmLtd
6146,VinDiesel PaulWalker JasonStatham MichelleRodr...,Action Thriller Crime,JamesWan,OriginalFilm OneRace UniversalPictures
